In [1]:
import faiss
import random
import torch
import numpy as np
import torch.nn.functional as F

from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
from datasets import load_from_disk, Dataset
from transformers import (
    AutoTokenizer,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments, RobertaModel, RobertaPreTrainedModel, BertModel, BertPreTrainedModel
)


In [2]:
class DenseRetrieval_with_Faiss:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        q_encoder,
        p_encoder=None,
        num_neg=5,
        hard_neg=1,
        is_trained=False,
    ):
        self.args = args
        self.dataset = dataset
        self.num_neg = num_neg

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder
        
        if torch.cuda.is_available():
            if p_encoder is not None:
                self.p_encoder.cuda()
            self.q_encoder.cuda()
        
        self.wiki_dataset = load_from_disk("/home/ubuntu/workspace/data/wiki_preprocessed_droped")
        
        if is_trained:
            pass
        else:
            self.prepare_in_batch_negative(num_neg=num_neg, hard_neg=hard_neg)

    def prepare_in_batch_negative(self,
        dataset=None,
        num_neg=4,
        hard_neg=1,
        k=100,
        tokenizer=None
    ):
        if num_neg < hard_neg:
            raise 'num_neg는 hard_neg보다 커야합니다.'
        wiki_datasets = self.wiki_dataset
        wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")
        if dataset is None:
            dataset = self.dataset

        if tokenizer is None:
            tokenizer = self.tokenizer

        # 1. In-Batch-Negative 만들기
        p_with_neg = []

        for c in tqdm(dataset):
            p_with_neg.append(c['context'])
            query = c['question']
            p_neg = []
            _, retrieved_examples = wiki_datasets.get_nearest_examples("text", query, k=k)
            for index in range(k):
                if retrieved_examples['document_id'][index] == c['document_id']:
                    continue
                p_neg.append(retrieved_examples['text'][index])
            p_with_neg.extend(p_neg[5:5+hard_neg])
            p_with_neg.extend(random.sample(p_neg[50:], num_neg - hard_neg))
            assert len(p_with_neg) % (num_neg + 1) == 0, '데이터가 잘못 추가되었습니다.'

        # 2. (Question, Passage) 데이터셋 만들어주기
        q_seqs = tokenizer(
            dataset["question"],
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        p_seqs = tokenizer(
            p_with_neg,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        max_len = p_seqs["input_ids"].size(-1)
        p_seqs["input_ids"] = p_seqs["input_ids"].view(-1, num_neg+1, max_len)
        p_seqs["attention_mask"] = p_seqs["attention_mask"].view(-1, num_neg+1, max_len)
        p_seqs["token_type_ids"] = p_seqs["token_type_ids"].view(-1, num_neg+1, max_len)

        train_dataset = TensorDataset(
            p_seqs["input_ids"], p_seqs["attention_mask"], p_seqs["token_type_ids"], 
            q_seqs["input_ids"], q_seqs["attention_mask"], q_seqs["token_type_ids"]
        )

        self.train_dataloader = DataLoader(
            train_dataset,
            shuffle=True,
            batch_size=self.args.per_device_train_batch_size
        )

    def build_faiss(self, del_p_encoder=True):
        eval_batch_size = 8

        self.search_corpus = list(set([example['text'] for example in self.wiki_dataset]))[:100]
        
        # Construt dataloader
        valid_p_seqs = self.tokenizer(
            self.search_corpus,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        valid_dataset = TensorDataset(
            valid_p_seqs["input_ids"],
            valid_p_seqs["attention_mask"],
            valid_p_seqs["token_type_ids"]
        )
        valid_sampler = SequentialSampler(valid_dataset)
        valid_dataloader = DataLoader(
            valid_dataset,
            sampler=valid_sampler,
            batch_size=eval_batch_size
        )

        # Inference using the passage encoder to get dense embeddeings
        p_embs = []

        with torch.no_grad():

            epoch_iterator = tqdm(
                valid_dataloader,
                desc="Iteration",
                position=0,
                leave=True
            )
            self.p_encoder.eval()

            for _, batch in enumerate(epoch_iterator):
                batch = tuple(t.cuda() for t in batch)

                p_inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2]
                }
                
                outputs = self.p_encoder(**p_inputs).to("cpu").numpy()
                p_embs.extend(outputs)
                
        # 이제 p encoder 쓸 일이 없을경우 삭제        
        if del_p_encoder:
            del self.p_encoder
            
        p_embs = np.array(p_embs)
        emb_dim = p_embs.shape[-1]

        cpu_index = faiss.IndexFlatL2(emb_dim)  # Flat에 GPU 사용
        self.indexer = faiss.index_cpu_to_all_gpus(cpu_index)
        self.indexer.add(p_embs)
        faiss.write_index(faiss.index_gpu_to_cpu(self.indexer), 'wiki.index')

    def train(self, args=None):
        if args is None:
            args = self.args
        batch_size = args.per_device_train_batch_size

        # Optimizer
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.adam_epsilon
        )
        t_total = len(self.train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=t_total
        )

        # Start training!
        self.p_encoder.train()
        self.q_encoder.train()
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = tqdm(range(int(args.num_train_epochs)), desc="Epoch")
        # for _ in range(int(args.num_train_epochs)):
        for _ in train_iterator:

            with tqdm(self.train_dataloader, unit="batch") as tepoch:
                for batch in tepoch:
            
                    targets = torch.zeros(batch_size).long() # positive example은 전부 첫 번째에 위치하므로
                    targets = targets.to(args.device)

                    p_inputs = {
                        "input_ids": batch[0].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "attention_mask": batch[1].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "token_type_ids": batch[2].view(batch_size * (self.num_neg + 1), -1).to(args.device)
                    }
            
                    q_inputs = {
                        "input_ids": batch[3].to(args.device),
                        "attention_mask": batch[4].to(args.device),
                        "token_type_ids": batch[5].to(args.device)
                    }

                    # (batch_size*(num_neg+1), emb_dim)
                    p_outputs = self.p_encoder(**p_inputs)
                    # (batch_size*, emb_dim)
                    q_outputs = self.q_encoder(**q_inputs)

                    # Calculate similarity score & loss
                    p_outputs_t = torch.transpose(p_outputs.view(batch_size, self.num_neg + 1, -1), 1 , 2)
                    q_outputs = q_outputs.view(batch_size, 1, -1)

                    sim_scores = torch.bmm(q_outputs, p_outputs_t).squeeze()  #(batch_size, num_neg + 1)
                    sim_scores = sim_scores.view(batch_size, -1)
                    sim_scores = F.log_softmax(sim_scores, dim=1)

                    loss = F.nll_loss(sim_scores, targets)
                    tepoch.set_postfix(loss=f"{str(loss.item())}")

                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                    self.p_encoder.zero_grad()
                    self.q_encoder.zero_grad()

                    global_step += 1

                    torch.cuda.empty_cache()

                    del p_inputs, q_inputs

    def get_relevant_doc(self, query, k=1):
        valid_q_seqs = self.tokenizer(query, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            self.q_encoder.eval()
            q_emb = self.q_encoder(**valid_q_seqs).to("cpu").numpy()
            del valid_q_seqs
        
        q_emb = q_emb.astype(np.float32)
        D, I = self.indexer.search(q_emb, k)
        distances, index = D.tolist()[0], I.tolist()[0]
        
        distance_list, doc_list = [], []
        for d, i in zip(distances, index):
            distance_list.append(d)
            doc_list.append(self.search_corpus[i])

        return distance_list, doc_list, D, I

In [3]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [4]:
class RobertaEncoder(RobertaPreTrainedModel):
    def __init__(self, config):
        super(RobertaEncoder, self).__init__(config)

        self.roberta = RobertaModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [5]:
# 평가를 안하니 검증데이터와 훈련데이터를 합칩니다.
train_dataset = load_from_disk('/home/ubuntu/workspace/data/train_dataset')
train = train_dataset['train'].to_dict()
valid = train_dataset['validation'].to_dict()
for key in train.keys():
  train[key].extend(valid[key])
train_dataset = Dataset.from_dict(train)
train_dataset = train_dataset.select(list(range(50)))
train_dataset

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'chunks'],
    num_rows: 50
})

In [6]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.seq_relationsh

In [7]:
retriever = DenseRetrieval_with_Faiss(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder
)

  0%|          | 0/50 [00:00<?, ?it/s]

/home/ubuntu/workspace/mrc_venv/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [8]:
retriever.train()

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?batch/s]

  0%|          | 0/25 [00:00<?, ?batch/s]

  0%|          | 0/25 [00:00<?, ?batch/s]

In [9]:
retriever.build_faiss()

Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

In [10]:
# 테스트
query = "실잠자리는 무엇을하는가?"

In [11]:
results = retriever.get_relevant_doc(query=query, k=5)

In [12]:
results[1]

['비록 흑점 발생의 세부사항은 아직 연구의 대상이지만 분명히 흑점은 차등회전에 의해 시작된 태양의 대류층 안의 자기 플럭스 튜브의 가시적인 카운터파트이다. 만약 플럭스 투브 위의 압력이 특정 한계에 도달하면 그들은 고무 밴드와 같이 말아올라가고 태양 표면을 뚫는다. 펑크점에서 대류는 막히고 태양 내부에서 나오는 에너지 흐름은 감소하며 그에 따라 표면 온도도 떨어진다. 윌슨 효과는 우리에게 흑점이 실제로는 태양 표면위의 저기압(depression)이라고 말해 준다. 쉽게 설명하자면 태양의 자전속도가 빨라지면서 태양의 자기장이 영향을 받아, 꼬이고 엉키면서 한 지점에서 집중적으로 자기장이 강한 부분이 생겨나게 되고, 강한 자기장으로 인해 태양의 대류가 지체가 되고 온도가 낮아지면서 흑점이 생겨나는 것이다.',
 '변광성 일반 목록에는 장주기 변광성이 정의되어 있지 않지만, 미라형 변광성은 주기가 긴 변광성으로 서술되어 있다 이 분류는 처음에 주기가 몇백 일 단위로 매우 긴 변광성을 분류하기 위해 사용되었다 20세기 중반에, 장주기 변광성의 정체가 차가운 거성으로 거의 확정되었고 미라형 변광성과 가까운 반규칙 변광성 등 변광성 전체에 대해 다시금 연구가 이루어져 "장주기 변광성"이라는 분류가 생겨나게 되었다. 반규칙 변광성은 장주기 변광성과 세페이드 변광성의 중간으로 여겨진다 변광성 일반 목록 출판 후, 미라형 및 반규칙 변광성(중 SRa)은 간혹 장주기 변광성으로 간주되었다 장주기 변광성은 넓게 보면 미라형, 반규칙, 저속 불규칙 변광성, OGLE 소진폭 적색거성(OSARGs)으로 볼 수 있지만 OSARGs는 일반적으로 장주기 변광성으로 취급되지 않으며 연구자 대부분은 장주기 변광성을 미라형 및 반규칙, 또는 미라형만으로 보고 있다 미국 변광성 관측자 협회의 장주기 변광성 문단에서는 "미라형, 반규칙, 황소자리 RV형 등 모든 적색거성"들을 다루고 있다 황소자리 RV형 변광성에는 SRc형(반규칙)과 Lc형(불규칙) 적색 거성이 포함된다. 최근 연구에서는 점근거성

In [13]:
p_encoder.save_pretrained('model')

In [ ]:
args.output

In [14]:
tokenizer.save_pretrained('model')

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')